In [1]:
import argparse
import yaml
import os
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

# Read config

In [2]:
config_path = 'configs/celebA_Young_Classifier.yaml'
with open(config_path) as f:
    config = yaml.safe_load(f)
    
for k in config.keys():
    print(k, config[k])

image_dir ./data/CelebA/images/
image_label_dict ./data/CelebA/Young_binary_classification.txt
log_dir ./output/classifier
name CelebA-Young
num_channel 3
num_class 1
input_size 128
batch_size 32
epochs 5
ckpt_dir_continue 


In [3]:
output_dir = os.path.join(config['log_dir'], config['name'], 'classifier_output')
print('Output directory: {}'.format(output_dir))

Output directory: ./output/classifier/CelebA-Young/classifier_output


In [4]:
data_df = pd.read_csv(config['image_label_dict'])
categories = data_df.columns[1: ].to_list()
print('Categories: {}'.format(categories))

Categories: ['Young']


# Load saved output from classifier

In [5]:
# Read classifier output
train_or_test = 'train'
names = np.load(os.path.join(output_dir, 'name_' + train_or_test+ '.npy'))
prediction_y = np.load(os.path.join(output_dir, 'prediction_y_' + train_or_test + '.npy'))
true_y = np.load(os.path.join(output_dir, 'true_y_' + train_or_test + '.npy'))
train_or_test = 'test'
names_i = np.load(os.path.join(output_dir,'name_' + train_or_test + '.npy'))
prediction_y_i = np.load(os.path.join(output_dir, 'prediction_y_' + train_or_test+'.npy'))
true_y_i = np.load(os.path.join(output_dir, 'true_y_' + train_or_test + '.npy'))

print(names.shape, prediction_y.shape, true_y.shape)
print(names_i.shape, prediction_y_i.shape, true_y_i.shape)

FileNotFoundError: [Errno 2] No such file or directory: './output/classifier/CelebA-Young/classifier_output/name_train.npy'

In [ ]:
true_y = np.reshape(true_y, [-1,len(categories)])
true_y_i = np.reshape(true_y_i, [-1,len(categories)])

# View results from classifier

In [ ]:
for i in range(prediction_y.shape[1]):
    print(categories[i],i)
    if prediction_y.shape[1] == 2:
        j = 1
    else:
        j = i
    print("ROC-AUC train: ", roc_auc_score(true_y[:,i], prediction_y[:,j]))
    try:
        print("ROC-AUC test: ", roc_auc_score(true_y_i[:,i], prediction_y_i[:,j]))
    except:
        a = 0
    print("Accuracy train: ", accuracy_score(true_y[:,i], (prediction_y[:,j]>0.5).astype(int)))
    print("Accuracy test: ", accuracy_score(true_y_i[:,i], (prediction_y_i[:,j]>0.5).astype(int)))
    print("Recall train: ", recall_score(true_y[:,i], (prediction_y[:,j]>0.5).astype(int)))
    print("Recall test: ", recall_score(true_y_i[:,i], (prediction_y_i[:,j]>0.5).astype(int)))
    tp = np.sum((prediction_y[true_y[:,i] == 1,j]>0.5).astype(int))
    tp_i = np.sum((prediction_y_i[true_y_i[:,i] == 1,j]>0.5).astype(int))
    print("Stats train: ", np.unique(true_y[:,i], return_counts=True), tp)
    print("Stats test: ", np.unique(true_y_i[:,i], return_counts=True),tp_i)
    print(confusion_matrix(true_y_i[:,i],  (prediction_y_i[:,j]>0.5).astype(int)))
    print(confusion_matrix(true_y[:,i],  (prediction_y[:,j]>0.5).astype(int)))
    if prediction_y.shape[1] == 2:
        break

# Choose a target class to build explanation

In [ ]:
current_index = 0
name = categories[current_index]
print(name)
current_index_prob = 1

# Create a dataframe from results

In [ ]:
if len(categories) == 1:
    df_test_results = pd.DataFrame(data = [names_i, true_y_i[:, 0], prediction_y_i[:, 0]]).T
else:
    df_test_results = pd.DataFrame(data = [names_i, true_y_i[:,current_index], prediction_y_i[:,current_index_prob]]).T

df_test_results = df_test_results.rename(index=str, columns={0: "filename", 1: "label", 2:"prob"})
df_test_results['bin'] = np.floor(df_test_results["prob"].astype('float') * 10).astype('int')
print(df_test_results.shape)
print(np.unique(df_test_results['bin'] ,return_counts=True))
df_test_results.head(3)

In [ ]:
if len(categories) == 1:
    df_train_results = pd.DataFrame(data = [names, true_y[:, 0], prediction_y[:, 0]]).T
else:
    df_train_results = pd.DataFrame(data = [names, true_y[:,current_index], prediction_y[:,current_index_prob]]).T

df_train_results = df_train_results.rename(index=str, columns={0: "filename", 1: "label", 2:"prob"})
df_train_results['bin'] = np.floor(df_train_results["prob"].astype('float') * 10).astype('int')
print(df_train_results.shape)
print(np.unique(df_train_results['bin'] ,return_counts=True))
df_train_results.head(3)

In [ ]:
df = pd.concat([df_train_results, df_test_results])
df = df.drop_duplicates()
print(df.shape)
df.head(1)

# Plot reliability curve

In [ ]:
import matplotlib.pyplot as plt

#Reliability Curve
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.metrics import (brier_score_loss, precision_score, recall_score,
                             f1_score)
true_label = np.asarray(df['label']).astype(int)
predicted_prob = np.asarray(df["prob"]).astype(float)
fraction_of_positives, mean_predicted_value = calibration_curve(true_label, predicted_prob, n_bins=10)
clf_score = brier_score_loss(true_label, predicted_prob, pos_label=1)
plt.plot(mean_predicted_value, fraction_of_positives, "s-",
                 label="%s (%1.3f)" % ('Data-before binning', clf_score))
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
plt.ylabel('Fraction of positives')
#plt.ylim([-0.05, 1.05])
plt.title('Calibration plots  (reliability curve) for '+ name)
plt.legend()
plt.show()

# Create bins

In [ ]:
df.head(1)

In [ ]:
df.groupby(by='bin').count()

In [ ]:
n = 5000

# Calibrated Sampling

In [ ]:
for i in range(10):
    print(i)
    df_bin = df.loc[df['bin'] == i]
    print(df_bin.shape)
    print(np.min(df_bin['prob']), np.max(df_bin['prob']))
    print(np.unique(df_bin['label'],return_counts=True))
    df_bin_0 = df.loc[(df['bin'] == i) & (df['label'] ==0)]
    df_bin_1 = df.loc[(df['bin'] == i) & (df['label'] ==1)]
    n_0 = int((1 - (0.1 * i) ) * n)
    if df_bin_0.shape[0] >= n_0:
        df_bin = df_bin_0.sample(n=n_0)
    else:
        df_bin = df_bin_0
        n_0 = df_bin_0.shape[0]
    n_1 = n - n_0
    if df_bin_1.shape[0] >= n_1:
        df_bin = pd.concat([df_bin, df_bin_1.sample(n=n_1)])        
    else:
        df_bin = pd.concat([df_bin, df_bin_1])    
        
    if i == 0:
        df_bin_all = df_bin
    else:
        df_bin_all = pd.concat([df_bin, df_bin_all])
    print(df_bin_all.shape)
    print(np.unique(df_bin['label'],return_counts=True))

In [ ]:
print(np.unique(df_bin_all['bin'] ,return_counts=True))

In [ ]:
#Reliability Curve
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.metrics import (brier_score_loss, precision_score, recall_score,
                             f1_score)
true_label = np.asarray(df_bin_all['label']).astype(int)
predicted_prob = np.asarray(df_bin_all['prob']).astype(float)
fraction_of_positives, mean_predicted_value = calibration_curve(true_label, predicted_prob, n_bins=10)
clf_score = brier_score_loss(true_label, predicted_prob, pos_label=1)
plt.plot(mean_predicted_value, fraction_of_positives, "s-",
                 label="%s (%1.3f)" % ('Data-after binning', clf_score))
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
plt.ylabel('Fraction of positives')
plt.ylim([-0.05, 1.05])
plt.title('Calibration plots  (reliability curve) for '+name)
plt.legend()
plt.show()

# Save the output

In [ ]:
experiment_dir = os.path.join('.'+config['log_dir'], config['name'], 'explainer_input')

In [ ]:
print(experiment_dir,name)
if not os.path.exists(experiment_dir):
    os.makedirs(experiment_dir)

In [ ]:
df_temp = df_bin_all[['filename', 'bin']]
df_temp.to_csv(os.path.join(experiment_dir, 'list_attr_celeba_'+name+'.txt'), sep = ' ', index = None, header = None)
print(df_temp.shape)
one_line = str(df_temp.shape[0]) + '\n'
second_line = "0-0.09 0.1-0.19 0.2-0.29 0.3-0.39 0.4-0.49 0.5-0.59 0.6-0.69 0.7-0.79 0.8-0.89 0.9-0.99\n"
with open(os.path.join(experiment_dir, 'list_attr_celeba_'+name+'.txt'), 'r+') as fp:
    lines = fp.readlines()     # lines is list of line, each element '...\n'
    lines.insert(0, one_line)  # you can use any index if you know the line index
    lines.insert(1, second_line)
    fp.seek(0)                 # file pointer locates at the beginning to write the whole file again
    fp.writelines(lines) 
fp = open(os.path.join(experiment_dir, 'list_attr_celeba_'+name+'.txt'), 'r+')
print(fp.readline())
print(fp.readline())
print(fp.readline())
print(fp.readline())
print(fp.readline())
print(fp.readline())
df_bin_all.to_csv(os.path.join(experiment_dir, 'Data_Output_Classifier_'+name+'.csv'), sep = ' ', index = None)
df_test_results.to_csv(os.path.join(experiment_dir, 'Data_Output_Classifier_All_Test_'+name+'.csv'), sep = ' ', index = None)
df_train_results.to_csv(os.path.join(experiment_dir, 'Data_Output_Classifier_All_Train_'+name+'.csv'), sep = ' ', index = None)

In [ ]:
df_combine = pd.read_csv(os.path.join(experiment_dir, 'Data_Output_Classifier_'+name+'.csv'), sep = ' ')

In [ ]:
#Reliability Curve
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.metrics import (brier_score_loss, precision_score, recall_score,
                             f1_score)

fraction_of_positives, mean_predicted_value = calibration_curve(df_combine['label'], df_combine["prob"], n_bins=10)
clf_score = brier_score_loss(df_combine['label'], df_combine["prob"], pos_label=1)
plt.plot(mean_predicted_value, fraction_of_positives, "s-",
                 label="%s (%1.3f)" % ('Data', clf_score))
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
plt.ylabel('Fraction of positives')
plt.ylim([-0.05, 1.05])
plt.title('Calibration plots  (reliability curve)')
plt.legend()
plt.show()